# Building Scotch Recommendation API

In [1]:
import pandas as pd
import pickle
import requests
import json

In [3]:
features_df = pd.read_pickle("/srv/notebooks/features.dataframe")

In [5]:
sim_df = pd.read_pickle("/srv/notebooks/sims.dataframe")

In [6]:
features_df = features_df.drop('cluster', axis=1)

In [7]:
REQUEST = json.dumps({
    'path' : {},
    'args' : {}
})

In [8]:
# GET /scotches
names = sim_df.columns.tolist()
print(json.dumps(dict(names=names)))

{"names": ["Aberfeldy", "Aberlour", "AnCnoc", "Ardbeg", "Ardmore", "ArranIsleOf", "Auchentoshan", "Auchroisk", "Aultmore", "Balblair", "Balmenach", "Belvenie", "BenNevis", "Benriach", "Benrinnes", "Benromach", "Bladnoch", "BlairAthol", "Bowmore", "Bruichladdich", "Bunnahabhain", "Caol Ila", "Cardhu", "Clynelish", "Craigallechie", "Craigganmore", "Dailuaine", "Dalmore", "Dalwhinnie", "Deanston", "Dufftown", "Edradour", "GlenDeveronMacduff", "GlenElgin", "GlenGarioch", "GlenGrant", "GlenKeith", "GlenMoray", "GlenOrd", "GlenScotia", "GlenSpey", "Glenallachie", "Glendronach", "Glendullan", "Glenfarclas", "Glenfiddich", "Glengoyne", "Glenkinchie", "Glenlivet", "Glenlossie", "Glenmorangie", "Glenrothes", "Glenturret", "Highland Park", "Inchgower", "Isle of Jura", "Knochando", "Lagavulin", "Laphroig", "Linkwood", "Loch Lomond", "Longmorn", "Macallan", "Mannochmore", "Miltonduff", "Mortlach", "Oban", "OldFettercairn", "OldPulteney", "RoyalBrackla", "RoyalLochnagar", "Scapa", "Speyburn", "Speys

In [9]:
# GET /scotches/:scotch
request = json.loads(REQUEST)
name = request['path'].get('scotch', 'Talisker')
features = features_df.loc[name]
# can't use to_dict because it retains numpy types which blow up when we json.dumps
print('{"features":%s}' % features.to_json())

{"features":{"Body":4,"Sweetness":2,"Smoky":3,"Medicinal":3,"Tobacco":0,"Honey":1,"Spicy":3,"Winey":0,"Nutty":1,"Malty":2,"Fruity":2}}


In [10]:
# GET /scotches/:scotch/similar
request = json.loads(REQUEST)
name = request['path'].get('scotch', 'Talisker')
count = request['args'].get('count', 5)
inc_features = request['args'].get('include_features', True)

similar = sim_df[name].sort_values(ascending=False)
similar.name = 'Similarity'
df = pd.DataFrame(similar).ix[1:count+1]

if inc_features:
    df = df.join(features_df)
    
df = df.reset_index().rename(columns={'Distillery': 'Name'})
result = {
    'recommendations' : [row[1].to_dict() for row in df.iterrows()],
    'for': name
}
print(json.dumps(result))

{"recommendations": [{"Similarity": 0.9605849651931131, "Fruity": 2, "Name": "Clynelish", "Tobacco": 1, "Malty": 1, "Medicinal": 3, "Honey": 0, "Nutty": 1, "Body": 3, "Winey": 0, "Smoky": 3, "Sweetness": 2, "Spicy": 2}, {"Similarity": 0.9484168381883901, "Fruity": 1, "Name": "Ardbeg", "Tobacco": 0, "Malty": 2, "Medicinal": 4, "Honey": 0, "Nutty": 1, "Body": 4, "Winey": 0, "Smoky": 4, "Sweetness": 1, "Spicy": 2}, {"Similarity": 0.8992710312446166, "Fruity": 1, "Name": "Caol Ila", "Tobacco": 1, "Malty": 1, "Medicinal": 2, "Honey": 0, "Nutty": 2, "Body": 3, "Winey": 0, "Smoky": 4, "Sweetness": 1, "Spicy": 2}, {"Similarity": 0.8695980950674561, "Fruity": 1, "Name": "Lagavulin", "Tobacco": 1, "Malty": 1, "Medicinal": 4, "Honey": 0, "Nutty": 1, "Body": 4, "Winey": 2, "Smoky": 4, "Sweetness": 1, "Spicy": 1}, {"Similarity": 0.839754059379345, "Fruity": 3, "Name": "Dalmore", "Tobacco": 0, "Malty": 2, "Medicinal": 1, "Honey": 1, "Nutty": 1, "Body": 3, "Winey": 2, "Smoky": 2, "Sweetness": 2, "Spi